<a href="https://colab.research.google.com/github/wesleycoutinhodev/b3-rag-qa-system/blob/main/v3_of_B3_RAG_QA_Colab_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B3 Q&A com RAG — versão com persistência no Google Drive

Inclui células para **baixar/restaurar** artefatos (corpus/chunks/meta/FAISS) de um **.zip público** no Google Drive e para **salvar/atualizar** o pacote após novo processamento.

## 1) Instalação de dependências

In [ ]:
!pip -q install transformers accelerate bitsandbytes sentence-transformers faiss-cpu unstructured pdfminer.six pypdf tiktoken tqdm gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.7 MB/s eta 0:00:00


## 2) Configurações e diretórios

In [ ]:
from pathlib import Path
DATA_DIR = Path("/content/data")
INDEX_DIR = Path("/content/index")
EVAL_DIR = Path("/content/eval")
for p in [DATA_DIR, INDEX_DIR, EVAL_DIR]:
    p.mkdir(exist_ok=True, parents=True)

MODEL_LLM = "Qwen/Qwen2.5-7B-Instruct"
EMB_MODEL = "BAAI/bge-m3"
RERANK_MODEL = "BAAI/bge-reranker-v2-m3"

CHUNK_TOKENS = 800
CHUNK_OVERLAP = 100
TOPK = 20
TOPN = 5
MAX_NEW_TOKENS = 450

# -----------------------------------------------------------------
# ✅ FLAG DE CONTROLE
# True  = Baixar e restaurar índice PRONTO do Drive (pula chunking/embedding).
# False = Processar do zero a partir do corpus.jsonl (executa chunking/embedding).
# -----------------------------------------------------------------
RESTORE_FROM_DRIVE = True

## 3) Persistência via Google Drive

In [ ]:
# 🔽 Baixar/restaurar CORPUS pronto (JSONL) do Drive público
from pathlib import Path
import os, json

CORPUS_FILE_ID = "1dmHUXwes4rpGW9XAYCJUUKv5HRvgRgkl"
CORPUS_JSONL_PATH = DATA_DIR / "corpus.jsonl"

def download_corpus_to_data_dir(file_id: str, dest_path: Path) -> bool:

    # Garante que o diretório de destino existe
    dest_path.parent.mkdir(exist_ok=True, parents=True)

    url = f"https_drive.google.com/uc?id={file_id}" # (corrigido para evitar link)

    # ✅ Baixa diretamente para o caminho final
    !gdown --fuzzy "{url}" -O "{str(dest_path)}"

    ok = dest_path.exists() and dest_path.stat().st_size > 0
    if ok:
        print(f"✅ Corpus baixado com sucesso para: {dest_path}")
    else:
        print(f"❌ Falha ao baixar o corpus de file_id: {file_id}")
    return ok

corpus_downloaded = download_corpus_to_data_dir(CORPUS_FILE_ID, CORPUS_JSONL_PATH)

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=https_drive.google.com/uc?id=1dmHUXwes4rpGW9XAYCJUUKv5HRvgRgkl

but Gdown can't. Please check connections and permissions.
❌ Falha ao baixar o corpus de file_id: 1dmHUXwes4rpGW9XAYCJUUKv5HRvgRgkl


In [ ]:
# 🔽 Baixar/restaurar artefatos de um ZIP público no Drive

if RESTORE_FROM_DRIVE:
    print("Iniciando restauração dos artefatos (index, meta, chunks) do Drive...")
    FILE_ID = "1zqTf2D3DvYooqy0iqnUb32kUIOtzJd8f"
    ZIP_DEST = "/content/b3_rag_artifacts.zip"

    def restore_from_drive_public(file_id: str):
        if not file_id:
            print("Defina FILE_ID para tentar baixar do Drive público.")
            return False
        !gdown --id $file_id -O $ZIP_DEST
        # Use a flag -o (overwrite) e -j (junk paths)
        !unzip -o -j /content/b3_rag_artifacts.zip -d /content/index/

        # Limpa o zip baixado
        if os.path.exists(ZIP_DEST):
            os.remove(ZIP_DEST)

        return (INDEX_DIR/"faiss.index").exists() and (INDEX_DIR/"meta.jsonl").exists()

    restored = restore_from_drive_public(FILE_ID)
    print("Restaurado do Drive público ✅" if restored else "❌ Falha ao restaurar do Drive.")
else:
    print("Pulando restauração de artefatos (RESTORE_FROM_DRIVE=False).")
    restored = False

Iniciando restauração dos artefatos (index, meta, chunks) do Drive...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zqTf2D3DvYooqy0iqnUb32kUIOtzJd8f
To: /content/b3_rag_artifacts.zip
100% 1.96M/1.96M [00:00<00:00, 168MB/s]
Archive:  /content/b3_rag_artifacts.zip
  inflating: /content/index/chunks.jsonl  
  inflating: /content/index/faiss.index  
  inflating: /content/index/meta.jsonl  
Restaurado do Drive público ✅


In [ ]:
# ⬆️ Salvar/atualizar artefatos no seu Drive (minha conta)
SAVE_PATH = "/content/drive/MyDrive/b3_rag_public/b3_rag_artifacts.zip"

# from google.colab import drive
# drive.mount('/content/drive')

#!mkdir -p /content/drive/MyDrive/b3_rag_public

#print(f"Salvando artefatos de {INDEX_DIR} para {SAVE_PATH}...")
#!zip -r $SAVE_PATH $INDEX_DIR
#print("ZIP atualizado em:", SAVE_PATH)

## 5) Chunking (pule se já restaurou)

In [ ]:
if not RESTORE_FROM_DRIVE:
    print("Iniciando processo de Chunking (RESTORE_FROM_DRIVE=False)...")
    import tiktoken, json, re, gc
    from pathlib import Path
    from tqdm import tqdm

    # Tokenizador
    enc = tiktoken.get_encoding("cl100k_base")

    def ntokens(s: str) -> int:
        try:
            return len(enc.encode(s))
        except Exception:
            return len(s.split())

    def chunk_text(text: str, chunk_tokens=CHUNK_TOKENS, overlap_tokens=CHUNK_OVERLAP):
        paras = [p.strip() for p in re.split(r"\n\s*\n", text) if p.strip()]
        cur = ""
        # emite um chunk por vez
        for p in paras:
            if ntokens(cur) + ntokens(p) <= chunk_tokens:
                cur += ("\n\n" + p) if cur else p
            else:
                if cur:
                    yield cur
                tail = " ".join(cur.split()[-overlap_tokens:]) if cur else ""
                cur = (tail + " " + p).strip()
        if cur:
            yield cur

    # ✅ Garante que o input path é o correto
    in_path  = DATA_DIR / "corpus.jsonl"
    meta_out = INDEX_DIR / "meta.jsonl"
    chunk_out = INDEX_DIR / "chunks.jsonl"

    # Zera arquivos de saída
    meta_out.write_text("", encoding="utf-8")
    chunk_out.write_text("", encoding="utf-8")

    total_docs_est = sum(1 for _ in open(in_path, encoding="utf-8"))
    print(f"📄 Corpus com {total_docs_est:,} documentos\n")

    total_docs = 0
    total_chunks = 0

    with open(in_path, "r", encoding="utf-8") as fin, \
         open(meta_out, "a", encoding="utf-8") as fmeta, \
         open(chunk_out, "a", encoding="utf-8") as fchunk:

        for line in tqdm(fin, total=total_docs_est, desc="🔹 Chunking", unit="doc"):
            if not line.strip():
                continue
            d = json.loads(line)
            total_docs += 1
            chunk_id = 0
            for ch in chunk_text(d["text"], CHUNK_TOKENS, CHUNK_OVERLAP):
                fmeta.write(json.dumps({
                    "doc_id": d["id"],
                    "title": d["title"],
                    "url": d.get("url",""),
                    "chunk_id": chunk_id
                }, ensure_ascii=False) + "\n")
                fchunk.write(json.dumps({"text": ch}, ensure_ascii=False) + "\n")
                total_chunks += 1
                chunk_id += 1

            del d
            gc.collect()

    print(f"\n✅ Finalizado: {total_docs:,} documentos processados, {total_chunks:,} chunks gerados.")
else:
    print("Pulando Chunking (RESTORE_FROM_DRIVE=True).")

Pulando Chunking (RESTORE_FROM_DRIVE=True).


## 6) Embeddings + FAISS (pule se já restaurou)

In [ ]:
if not RESTORE_FROM_DRIVE:
    print("Iniciando processo de Embeddings e FAISS (RESTORE_FROM_DRIVE=False)...")
    from sentence_transformers import SentenceTransformer
    import numpy as np, faiss, json
    from tqdm import tqdm

    emb = SentenceTransformer(EMB_MODEL)
    chunks = [json.loads(l)["text"] for l in open(INDEX_DIR/"chunks.jsonl", encoding="utf-8")]
    meta = [json.loads(l) for l in open(INDEX_DIR/"meta.jsonl", encoding="utf-8")]

    vecs = []
    for ch in tqdm(chunks, desc="Embeddings"):
        v = emb.encode(ch, convert_to_numpy=True, normalize_embeddings=True)
        vecs.append(v.astype("float32"))

    vectors = np.vstack(vecs).astype("float32")

    index = faiss.IndexFlatIP(vectors.shape[1])
    index.add(vectors)
    faiss.write_index(index, str(INDEX_DIR/"faiss.index"))
    print("Index size:", index.ntotal)
else:
    print("Pulando Embeddings/FAISS (RESTORE_FROM_DRIVE=True).")

Pulando Embeddings/FAISS (RESTORE_FROM_DRIVE=True).


In [ ]:
!ls /content/index

chunks.jsonl  faiss.index  meta.jsonl


## 7) Busca e (opcional) reranker

In [ ]:
import json, numpy as np, faiss
index = faiss.read_index(str(INDEX_DIR/"faiss.index"))
meta = [json.loads(l) for l in open(INDEX_DIR/"meta.jsonl", encoding="utf-8")]
chunks = [json.loads(l)["text"] for l in open(INDEX_DIR/"chunks.jsonl", encoding="utf-8")]

from sentence_transformers import SentenceTransformer
emb = SentenceTransformer(EMB_MODEL)

def search(query: str, topk=TOPK):
    qv = emb.encode(query, convert_to_numpy=True, normalize_embeddings=True).astype("float32")[None,:]
    D, I = index.search(qv, topk)
    cands = [{"text": chunks[i], **meta[i], "score": float(D[0][j])} for j,i in enumerate(I[0])]
    return cands

try:
    from sentence_transformers import CrossEncoder
    reranker = CrossEncoder(RERANK_MODEL)
    def rerank(query, cands, topn=TOPN):
        pairs = [(query, c["text"]) for c in cands]
        scores = reranker.predict(pairs).tolist()
        for c, s in zip(cands, scores):
            c["rerank"] = float(s)
        return sorted(cands, key=lambda x: x["rerank"], reverse=True)[:topn]
except Exception as e:
    print("[INFO] Reranker indisponível:", e)
    reranker = None
    def rerank(query, cands, topn=TOPN):
        return cands[:topn]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

## 8) LLM 4-bit e Q&A

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print("Carregando LLM (Qwen) e Tokenizer...")
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
tok = AutoTokenizer.from_pretrained(MODEL_LLM)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_LLM, device_map="auto", quantization_config=bnb
)

def build_prompt(question, ctx_items):
    # ✅ Lógica de prompt mantida
    ctx_str = "\n\n".join(
        [f"[{i+1}] Título: {c['title']}\nFonte: {c.get('url','')}\nTrecho:\n{c['text'][:1800]}"
         for i,c in enumerate(ctx_items)]
    )
    return (
        "Você é um assistente financeiro e responde sobre a B3.\n"
        "REGRAS: Use APENAS as informações do CONTEXTO. Se faltar evidência, diga que não sabe.\n"
        "Inclua referências [nº] e liste as fontes no final.\n\n"
        f"PERGUNTA:\n{question}\n\n"
        f"CONTEXTO:\n{ctx_str}\n\n"
        "RESPOSTA:"
    )

@torch.inference_mode()
def answer(question, topk=TOPK, topn=TOPN, max_new_tokens=MAX_NEW_TOKENS, temperature=0.0):
    # ✅ Funções 'search' e 'rerank' foram definidas na Célula 22
    cands = search(question, topk=topk)
    ctx = rerank(question, cands, topn=topn)
    prompt = build_prompt(question, ctx)
    input_ids = tok(prompt, return_tensors="pt").to(model.device)
    out_ids = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False if temperature==0 else True,
        temperature=temperature,
        pad_token_id=tok.eos_token_id
    )
    text = tok.decode(out_ids[0], skip_special_tokens=True)
    resp = text.split("RESPOSTA:")[-1].strip()
    fontes = "\n".join([f"[{i+1}] {c['title']} — {c.get('url','')}" for i,c in enumerate(ctx)])
    return resp + "\n\nFontes:\n" + fontes

print("✅ LLM e função 'answer' prontos.")

## 9) Teste rápido

In [ ]:
print(answer("Como a taxa Selic afetou o mercado financeiro?", topn=4))

In [ ]:
import numpy as np
import faiss

# --- Contexto Simulado (Valores reais viriam da execução do seu RAG) ---
# Simulação do vetor de embedding da pergunta (dim=1024 para bge-m3)
QUERY_EMBEDDING = np.random.rand(1, 1024).astype('float32')
QUERY_EMBEDDING /= np.linalg.norm(QUERY_EMBEDDING) # Normalização!

# Simulação da busca FAISS (Normalmente o FAISS retorna D=Distâncias e I=Índices)
# D: Matriz de distâncias (scores de similaridade). Como usamos IP e normalizamos, D é o Cosseno.
# I: Matriz de índices (ID do chunk no metadados/corpus).
D_SIMULADO = np.array([[0.98, 0.95, 0.92, 0.85, 0.70]]) # Cosseno alto = Alta similaridade
I_SIMULADO = np.array([[25, 42, 18, 99, 5]]) # IDs dos chunks
K_BUSCA = 5 # Top-K buscado

# Simulação dos metadados (para mapear o ID do chunk à sua origem e texto)
METADATA_CHUNKS = {
    25: {"texto_chunk": "A alta do Ibovespa foi impulsionada pelo fluxo estrangeiro...", "fonte": "Fonte A"},
    42: {"texto_chunk": "O corte de juros nos EUA atraiu capital para mercados emergentes...", "fonte": "Fonte B"},
    18: {"texto_chunk": "Empresas como Vale e Petrobras tiveram ganhos significativos...", "fonte": "Fonte A"},
    99: {"texto_chunk": "O Ibovespa atingiu novo recorde em 2025...", "fonte": "Fonte C"},
    5: {"texto_chunk": "A B3 lança novo índice de títulos...", "fonte": "Fonte D"},
}
# --- Fim do Contexto Simulado ---


def extrair_e_analisar_faiss_scores(D, I, metadata, query_embedding):
    """
    Função para analisar e estruturar as métricas FAISS recuperadas.

    Argumentos:
        D (np.array): Matriz de scores de similaridade (distâncias).
        I (np.array): Matriz de índices dos chunks recuperados.
        metadata (dict): Dicionário de metadados para mapear o índice ao conteúdo.
        query_embedding (np.array): Embedding da pergunta.
    """
    if D.shape[0] == 0:
        print("Nenhum resultado encontrado pelo FAISS.")
        return []

    # O FAISS retorna os scores, que, devido à normalização, SÃO as Distâncias de Cosseno.
    scores_cosseno = D[0]
    indices_chunks = I[0]

    resultados_detalhados = []

    print(f"--- Análise da Busca FAISS (Top-{len(indices_chunks)}) ---")
    print(f"Dimensão do Embedding da Pergunta: {query_embedding.shape[1]}")
    print(f"Tipo de Similaridade: Produto Interno (IP) em Vetores Normalizados = Distância de Cosseno\n")

    for rank, (score, chunk_id) in enumerate(zip(scores_cosseno, indices_chunks)):
        # Obter os metadados do chunk
        chunk_info = metadata.get(chunk_id, {"texto_chunk": "Chunk não encontrado", "fonte": "N/A"})

        resultado = {
            "rank": rank + 1,
            "score_cosseno": float(score),
            "chunk_id": int(chunk_id),
            "fonte": chunk_info['fonte'],
            "inicio_do_chunk": chunk_info['texto_chunk'][:100] + "..."
        }
        resultados_detalhados.append(resultado)

        # Imprimir o resumo
        print(f"[{rank + 1}º] Score de Cosseno: {score:.4f} (ID: {chunk_id})")
        print(f"    Fonte: {chunk_info['fonte']}")
        print(f"    Início do Texto: {chunk_info['texto_chunk'][:80]}...")
        print("-" * 40)

    # Métrica Agregada: Média do Top-K
    print(f"\nMédia de Similaridade do Top-{len(indices_chunks)}: {np.mean(scores_cosseno):.4f}")

    return resultados_detalhados

# Execução da análise (usando dados simulados)
analise_faiss = extrair_e_analisar_faiss_scores(D_SIMULADO, I_SIMULADO, METADATA_CHUNKS, QUERY_EMBEDDING)

# O output 'analise_faiss' pode ser usado para depuração ou para alimentar o Reranker (se ativo)
# print(analise_faiss)

In [ ]:
# Lista de perguntas para o teste final
perguntas_teste = [
    "O que impulsionou a alta do Ibovespa?",
    "Como a taxa Selic afetou o mercado financeiro?",
    "Quais setores apresentaram crescimento na bolsa?",
    "É um bom momento para investir em ações?",
    "O que se espera do plano estratégico da Petrobras?"
]

print("Iniciando Teste Final com", len(perguntas_teste), "perguntas...")
print("="*80)

# Executa cada pergunta
for i, pergunta in enumerate(perguntas_teste):
    print(f"\n--- TESTE {i+1}: {pergunta} ---\n")
    try:
        # Chama a função 'answer' definida na célula do LLM
        resposta = answer(pergunta, topn=4) # Usando topn=4 como no seu exemplo
        print(resposta)
    except Exception as e:
        print(f"Ocorreu um erro ao processar a pergunta: {e}")
    print("="*80)

print("\n✅ Teste final concluído.")